In [1]:
import pandas as pd
import numpy as np
import os
from glob import iglob

from datetime import datetime
import matplotlib.pyplot as plt

# my modules
import set_config
from common_functions import create_df_from_file


In [2]:
# call set_config
dir_sanofi_share = set_config.ConfigSectionMap("SectionOne")['sanofi']
dir_local = set_config.ConfigSectionMap("SectionOne")['local']

In [3]:
folder = 'IP21_data'

# call function with dir, folder, search criteria to find files, name of dataframe to create
# these are the bad pen counts
df_ORDER = create_df_from_file(dir_sanofi_share, folder, "_ORDER")
df_ORDER['IP_TREND_TIME'] = pd.to_datetime(df_ORDER['IP_TREND_TIME'], format='%d-%b-%y %H:%M:%S.%f')

C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/IP21_data\August\tabIP21Data_36630901_ORDERNUMBER.csv
C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/IP21_data\August\tabIP21Data_36630901_ZA_ORDERNUMBER.csv
C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/IP21_data\August\tabIP21Data_36650901_ORDERNUMBER.csv
C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/IP21_data\September\tabIP21Data_36630901_ORDERNUMBER.csv
C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/IP21_data\September\tabIP21Data_36630901_ZA_ORDERNUMBER.csv
C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/IP21_data\September\tabIP21Data_36650901_ORDERNUMBER.csv


In [6]:
names = df_ORDER.Name.unique()
names

array(['36630901_ORDERNUMBER', '36630901_ZA_ORDERNUMBER',
       '36650901_ORDERNUMBER'], dtype=object)

In [7]:
df_ORDER.groupby('Name')['IP_TREND_VALUE'].describe()

,count,unique,top,freq
Name,,,,
36630901_ORDERNUMBER,45.0,39.0,80764309.0,3.0
36630901_ZA_ORDERNUMBER,45,13,1F004,8
36650901_ORDERNUMBER,83.0,48.0,80760793.0,7.0


In [8]:
df_36630901_ORDERNUMBER = df_ORDER[(df_ORDER.Name == '36630901_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)].copy()
df_36650901_ORDERNUMBER = df_ORDER[(df_ORDER.Name == '36650901_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)].copy()
df_36630901_ZA_ORDERNUMBER = df_ORDER[(df_ORDER.Name == '36630901_ZA_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)].copy()

df_36630901_ORDERNUMBER.rename(columns={'IP_TREND_VALUE':'36630901_ORDERNUMBER'}, inplace=True)
df_36650901_ORDERNUMBER.rename(columns={'IP_TREND_VALUE':'36650901_ORDERNUMBER'}, inplace=True)
df_36630901_ZA_ORDERNUMBER.rename(columns={'IP_TREND_VALUE':'36630901_ZA_ORDERNUMBER'}, inplace=True)

merge = pd.merge(df_36630901_ORDERNUMBER[['IP_TREND_TIME','36630901_ORDERNUMBER']], df_36630901_ZA_ORDERNUMBER[['IP_TREND_TIME','36630901_ZA_ORDERNUMBER']], on='IP_TREND_TIME')
merge2 = pd.merge(merge, df_36650901_ORDERNUMBER[['IP_TREND_TIME','36650901_ORDERNUMBER']], on='IP_TREND_TIME', how='outer')

In [9]:
merge2.sort_values('IP_TREND_TIME', inplace=True)
merge2

,IP_TREND_TIME,36630901_ORDERNUMBER,36630901_ZA_ORDERNUMBER,36650901_ORDERNUMBER
10,2021-08-02 20:43:03.700,80760843.0,1F022,NaN
69,2021-08-02 22:51:44.100,NaN,NaN,80757084.0
68,2021-08-03 19:56:16.900,NaN,NaN,80756656.0
67,2021-08-04 00:59:12.800,NaN,NaN,80756289.0
9,2021-08-04 06:46:41.500,80760846.0,1F022,NaN
...,...,...,...,...
73,2021-09-29 21:00:03.800,NaN,NaN,80763694.0
72,2021-09-29 23:19:26.600,NaN,NaN,80763694.0
71,2021-09-29 23:20:40.000,NaN,NaN,80763694.0
11,2021-09-30 12:40:19.400,80765625.0,1F004,NaN


In [16]:
df_BATCHID = create_df_from_file(dir_sanofi_share, folder, "_BATCHID.csv")
df_BATCHID['IP_TREND_TIME'] = pd.to_datetime(df_BATCHID['IP_TREND_TIME'], format='%d-%b-%y %H:%M:%S.%f')

df_MATNO = create_df_from_file(dir_sanofi_share, folder, "_MATNO.csv")
df_MATNO['IP_TREND_TIME'] = pd.to_datetime(df_MATNO['IP_TREND_TIME'], format='%d-%b-%y %H:%M:%S.%f')

In [25]:
merge3 = pd.merge(merge2, df_BATCHID[['IP_TREND_TIME','IP_TREND_VALUE']], left_on='IP_TREND_TIME', right_on='IP_TREND_TIME', how='left')
merge3.rename(columns={'IP_TREND_VALUE':'BATCH_ID'}, inplace=True)

In [26]:
merge4 = pd.merge(merge3, df_MATNO[['IP_TREND_TIME','IP_TREND_VALUE']], left_on='IP_TREND_TIME', right_on='IP_TREND_TIME', how='left')
merge4.rename(columns={'IP_TREND_VALUE':'BATCH_SIZE'}, inplace=True)

In [27]:
merge4

,IP_TREND_TIME,36630901_ORDERNUMBER,36630901_ZA_ORDERNUMBER,36650901_ORDERNUMBER,BATCH_ID,BATCH_SIZE
0,2021-08-02 20:43:03.700,80760843.0,1F022,NaN,1F111,533064.0
1,2021-08-02 20:43:03.700,80760843.0,1F022,NaN,1F111,530092.0
2,2021-08-02 22:51:44.100,NaN,NaN,80757084.0,NaN,527537.0
3,2021-08-02 22:51:44.100,NaN,NaN,80757084.0,NaN,527537.0
4,2021-08-03 19:56:16.900,NaN,NaN,80756656.0,NaN,535797.0
...,...,...,...,...,...,...
270,2021-09-29 23:20:40.000,NaN,NaN,80763694.0,NaN,536578.0
271,2021-09-30 12:40:19.400,80765625.0,1F004,NaN,1F167,794304.0
272,2021-09-30 12:40:19.400,80765625.0,1F004,NaN,1F167,781598.0
273,2021-09-30 13:57:56.600,NaN,NaN,80763643.0,NaN,526133.0
